<center><img src="images/MLU-NEW-logo.png" alt="drawing" width="400" style="background-color:white; padding:1em;" /></center> <br/>


# <a name="0">Bedrock Agents-Based AI AppBuilder Assistant with Boto3 SDK </a>

## Notebook Overview

In this notebook, we build an agent to act as an Application Builder Assistant. Typically to build a three-tier software application, we need a user interface (UI component), middle tier (aka backend) for business APIs, and a database tier. With this assistant, we can generate SQL queries using a database schema (DDL: Data definition language) and execute them against a database instance, recommend software design best practices using the AWS Well-Architected Framework, and generate code snippets in many of the standard programming languages to improve developer productivity and facilitate rapid development of use-cases.

<div style="border: 4px solid coral; text-align: left; margin: auto; padding-left: 20px; padding-right: 20px">
    <h4>This notebook auto-cleans up resources to be frugal. </h4>
    You can visit this section (<a href="#10"> Clean-up Resources</a>) to change the setting if you need to experiment with prompts and settings. Please run clean-up resources after you are done with experiments. <br/>
</div>
<br/>


##### Notebook Kernel
Please choose `conda_python3` as the kernel type of the top right corner of the notebook if that does not appear by default.


## LLM Used
Anthropic Claude 2.0

## Use-Case Overview
Lets look at when this Application Builder Assistant acts as a coding assistant vs recommends AWS Design Best Practices vs aids in SQL code generation and execution.

First it is important to understand the type of task the user is asking for:

Does the user want to write and validate SQL Query against a database?
- If yes, use the existing DDL schemas to come up with the SQL query.

Does the user want to author some code for say, helper functions like validate email?
- If yes, use prompt engineering techniques to generate the email validation code.

Does the user want recommendation on design best practices?
- If yes, look up with AWS Well-Architected Knowledge Base.

<center><img src="images/agents.drawio.png" alt="architecture diagram for this notebook to demonstrate the conditional workflow for llms. This shows 3 workflows possible via this Application Builder Assistant. 1) Text to SQL - generate SQL statements via natural language and execute it against a local DB 2) web scraped knowledge base on AWS well architected framework - user can ask questions on it 3) Write and explain code via Claude LLM. User can ask any of these three types of questions making it an application builder assistant."  height="700" width="700" style="background-color:white; padding:1em;" /></center> <br/>



It demonstrates how to:
1. Select the underlying foundation model (FM) for your agent 
2. Provide a clear and concise agent instruction 
3. Create and associate an action group with an API schema and a Lambda function 
4  Create and associate multiple knowledge bases (2 KBs in this notebook) with the agent
5. Create, invoke, test, and deploy the agent
6. Generate UI/Backend code with LLMs
7. Recommend AWS best practices for system design with the AWS Well-Architected Framework guidelines
8. Generate and execute/validate the SQL from natural language understanding via LLMs, few shot examples, and a database schema as a knowledge base
9. Clean up resources (Be Frugal)


We are using the Retrieval Augmented Generation (RAG) technique with Amazon Bedrock. A RAG implementation consists of two parts:

    A data pipeline that ingests that from documents (typically stored in Amazon S3) into a knowledge base i.e. a vector database such as Amazon OpenSearch Service Serverless (AOSS) so that it is available for lookup when a question is received.

    An application that receives a question from the user, looks up the knowledge base for relevant pieces of information (context) and then creates a prompt that includes the question and the context and provides it to an LLM for generating a response.

The data pipeline represents an undifferentiated heavy lifting and can be implemented using Amazon Bedrock Agents for Knowledge Base. We can now connect an S3 bucket to a vector database such as AOSS and have a Bedrock Agent read the objects (html, pdf, text etc.), chunk them, and then convert these chunks into embeddings using Amazon Titan Embeddings model and then store these embeddings in AOSS. All of this without having to build, deploy, and manage the data pipeline.



Once the data is available in the Bedrock knowledge base then user questions can be answered using the following system design:


This notebook has the following sections:

1. <a href="#1">Environment configuration</a>
2. <a href="#2">Set up Bedrock for inference</a>
3. <a href="#3">Setup prefix variables for various agent resources</a>
4. <a href="#4">Creating an agent</a>
5. <a href="#5">Deploy agent and create agent alias</a>
6. <a href="#6">Invoke agent </a>
7. <a href="#7">Code generation examples</a>
8. <a href="#8">System design Q&A with the AWS Well-Architected Framework</a>
9. <a href="#9">SQL generation and execution against a database</a>
10. <a href="#10">Clean up resources</a>
    
Please work top to bottom of this notebook and don't skip sections as this could lead to error messages due to missing code.

---


Let's start by installing all required packages as specified in the `requirements.txt` file and importing several libraries.


## <a name="1">Environment setup and configuration</a>
(<a href="#0">Go to top</a>)

Before starting, let's import the required packages and configure the support variables:

In [1]:
%%capture
!pip3 install -r requirements.txt --quiet

In [2]:
import logging
import boto3
import random
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint
import os, shutil
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from fetch_aws_best_practices import *

from mlu_utils.agents_utils import *
from mlu_utils.agents_infra_utils_with_two_kbs import *
from mlu_utils.show_trace_widget import *
from mlu_utils.summarize_agent_trace import *
from IPython.display import Markdown

[2024-07-12 22:48:48,871] p28855 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [3]:
# cleanup trace files to avoid issues
trace_file_path = "./trace_files/"
if os.path.isdir(trace_file_path):
    shutil.rmtree(trace_file_path)
os.mkdir('trace_files')

In [4]:
clean_up_trace_files("./trace_files/")

In [5]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

pp = pprint.PrettyPrinter(width=41, compact=True)

In [6]:
import ipywidgets as widgets
from IPython.display import JSON
out = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))

out_3_tabs_1 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_tabs_2 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_tabs_3 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_tabs_4 = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))

out_3_turn_1_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_turn_2_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_turn_3_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))
out_3_turn_4_summary = widgets.Output(layout=widgets.Layout(border = '1px solid black', width = '100%',))


### <a name="2">2. Set up Bedrock for inference</a>
(<a href="#0">Go to top</a>)

To get started, set up Bedrock and instantiate an active `bedrock-runtime` to query LLMs. The code below leverages [LangChain's Bedrock integration](https://python.langchain.com/docs/integrations/llms/bedrock).
```
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

```

</br>

In [7]:
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
open_search_serverless_client = boto3.client('opensearchserverless')

# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')

session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '339712993987')

### <a name="3">3. Setup agent infrastructure</a>
(<a href="#0">Go to top</a>)

This is the same set of instructions for infrastructure setup as provided in Lab 2a and includes how to:
- Setup for prefix variables with various agent resources
- Create Lambda function for action group
- Create knowledge base 1 for SQL generation with Northwind Database
- Create knowledge base 2 for system design recommendation with AWS Well-Architected Framework
- Create an agent



In [8]:
%%time
infra_response = setup_agent_infrastructure()

[2024-07-12 22:48:49,710] p28855 {agents_infra_utils_with_two_kbs.py:55} INFO - random_uuid :: ecb7c079-dac5-4ec7-955f-d1a51173fb89 prefix_infra :: l2ecb7c0 prefix_iam :: l2dac5


Creating collection...
Creating collection...


[2024-07-12 22:51:16,377] p28855 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole



Collection successfully created:


[2024-07-12 22:52:01,971] p28855 {base.py:258} INFO - PUT https://odr4bp1mfekzisxzy01h.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-l2ecb7c0-kbdb-index [status:200 request:0.550s]


...


Collection successfully created:
[{'arn': 'arn:aws:aoss:us-east-1:339712993987:collection/0204jk1xap6898whg06e', 'collectionEndpoint': 'https://0204jk1xap6898whg06e.us-east-1.aoss.amazonaws.com', 'createdDate': 1720824834175, 'dashboardEndpoint': 'https://0204jk1xap6898whg06e.us-east-1.aoss.amazonaws.com/_dashboards', 'description': 'OpenSearch collection for Amazon Bedrock Knowledge Base for Northwind DB', 'id': '0204jk1xap6898whg06e', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1720824857772, 'name': 'l2dac5-kbaws-339712993987', 'standbyReplicas': 'DISABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}]


[2024-07-12 22:55:09,969] p28855 {base.py:258} INFO - PUT https://0204jk1xap6898whg06e.us-east-1.aoss.amazonaws.com:443/bedrock-knowledge-base-l2ecb7c0-kbaws-index [status:200 request:0.473s]



Creating index:


[2024-07-12 22:57:01,768] p28855 {agents_infra_utils_with_two_kbs.py:1226} INFO - agent_name :: l2ecb7c0-agent-kb 
 agent_alias_name :: l2ecb7c0-workshop-alias 
 agent_role :: {'Role': {'Path': '/', 'RoleName': 'AmazonBedrockExecutionRoleForAgents_l2dac5', 'RoleId': 'AROAU6GDY5LB4UOCZYA4G', 'Arn': 'arn:aws:iam::339712993987:role/AmazonBedrockExecutionRoleForAgents_l2dac5', 'CreateDate': datetime.datetime(2024, 7, 12, 22, 56, 41, tzinfo=tzlocal()), 'AssumeRolePolicyDocument': {'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'bedrock.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}}, 'ResponseMetadata': {'RequestId': '12fac478-1b55-4a57-8878-adf99800dd60', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 22:56:41 GMT', 'x-amzn-requestid': '12fac478-1b55-4a57-8878-adf99800dd60', 'content-type': 'text/xml', 'content-length': '839'}, 'RetryAttempts': 0}} 
 bucket_name :: l2ecb7c0-agent-kb-339712993987 
 schema_key :: l2ecb7c0-agent-kb-schema

CPU times: user 5.11 s, sys: 486 ms, total: 5.59 s
Wall time: 8min 12s


In [9]:
agent_name = infra_response["agent_name"]
agent_alias_name = infra_response["agent_alias_name"]
agent_role = infra_response["agent_role"]
bucket_name = infra_response["bucket_name"]
schema_key = infra_response["schema_key"]
knowledge_base_db_id = infra_response["knowledge_base_db_id"]
knowledge_base_aws_id = infra_response["knowledge_base_aws_id"]
lambda_name = infra_response["lambda_name"]
lambda_function = infra_response["lambda_function"]
agent_bedrock_policy = infra_response["agent_bedrock_policy"]
agent_s3_schema_policy = infra_response["agent_s3_schema_policy"]
agent_role_name = infra_response["agent_role_name"]
lambda_role_name = infra_response["lambda_role_name"]
kb_db_collection_name = infra_response["kb_db_collection_name"]
kb_db_bedrock_policy = infra_response["kb_db_bedrock_policy"]
kb_db_aoss_policy = infra_response["kb_db_aoss_policy"]
kb_db_s3_policy = infra_response["kb_db_s3_policy"]
agent_kb_schema_policy = infra_response["agent_kb_schema_policy"]
kb_db_role_name = infra_response["kb_db_role_name"]
kb_db_opensearch_collection_response = infra_response["kb_db_opensearch_collection_response"]
kb_aws_collection_name = infra_response["kb_aws_collection_name"]
kb_aws_bedrock_policy = infra_response["kb_aws_bedrock_policy"]
kb_aws_aoss_policy = infra_response["kb_aws_aoss_policy"]
kb_aws_s3_policy = infra_response["kb_aws_s3_policy"]
kb_aws_role_name = infra_response["kb_aws_role_name"]
kb_aws_opensearch_collection_response = infra_response["kb_aws_opensearch_collection_response"]


### <a name="3">Creating an agent </a>
(<a href="#0">Go to top</a>)

Once the needed IAM role is created, we can use the Bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model, and instruction. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs:

In [10]:
# Create Agent
agent_instruction = """
Hello, I am AI Application Builder Assistant. 

I am capable of answering the following three categories of questions:
- Best practices for design of software applications using the content inside the AWS best practices and AWS well-architected framework Knowledge Base. I help customers understand AWS best practices for building applications with AWS services.  
- Generate a valid SQLite query for the customer using the database schema inside the Northwind DB knowledge base and then execute the query that answers the question based on the [Northwind] dataset.If the Northwind DB Knowledge Base search function result did not contain enough information to construct a full query try to construct a query to the best of your ability based on the Northwind database schema.
- Generate and Explain code for the customer following standard programming language syntax 

Feel free to ask any questions along those lines!

"""
#foundationModel="anthropic.claude-3-sonnet-20240229-v1:0",
# "anthropic.claude-v2"

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="AI Application Builder Assistant",
    idleSessionTTLInSeconds=1800,
    foundationModel="anthropic.claude-v2",
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id:

In [11]:
response

{'ResponseMetadata': {'RequestId': '005fc141-5594-469f-b395-6ee89a8d8322',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 22:57:02 GMT',
   'content-type': 'application/json',
   'content-length': '1419',
   'connection': 'keep-alive',
   'x-amzn-requestid': '005fc141-5594-469f-b395-6ee89a8d8322',
   'x-amz-apigw-id': 'a0kttEmAIAMEDDg=',
   'x-amzn-trace-id': 'Root=1-6691b4bd-3300a5a843b7f50f53f5e677'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:339712993987:agent/WONPD0DVUU',
  'agentId': 'WONPD0DVUU',
  'agentName': 'l2ecb7c0-agent-kb',
  'agentResourceRoleArn': 'arn:aws:iam::339712993987:role/AmazonBedrockExecutionRoleForAgents_l2dac5',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 7, 12, 22, 57, 1, 995916, tzinfo=tzlocal()),
  'description': 'AI Application Builder Assistant',
  'foundationModel': 'anthropic.claude-v2',
  'idleSessionTTLInSeconds': 1800,
  'instruction': '\nHello, I am AI Application Builder 

Let's now store the agent id in a local variable to use it on the next steps

In [12]:
agent_id = response['agent']['agentId']
agent_id

'WONPD0DVUU'

### Create agent action group
We will now create an agent action group that uses the Lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [13]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='AppBuilderAssistantActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': schema_key
        }
    },
    description='APIs to execute valid SQLite query and help customers with system design by querying the AWS Well-Architected Framework and writing code.'
)

In [14]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '542df0e7-be7b-4f72-a845-21fd9b4da029',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 22:57:32 GMT',
   'content-type': 'application/json',
   'content-length': '649',
   'connection': 'keep-alive',
   'x-amzn-requestid': '542df0e7-be7b-4f72-a845-21fd9b4da029',
   'x-amz-apigw-id': 'a0kycGLloAMEXVw=',
   'x-amzn-trace-id': 'Root=1-6691b4dc-3e9433d4213946ba52c1c34d'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:339712993987:function:l2ecb7c0-agent-kb-339712993987'},
  'actionGroupId': 'FM01D7TEPW',
  'actionGroupName': 'AppBuilderAssistantActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'WONPD0DVUU',
  'agentVersion': 'DRAFT',
  'apiSchema': {'s3': {'s3BucketName': 'l2ecb7c0-agent-kb-339712993987',
    's3ObjectKey': 'l2ecb7c0-agent-kb-schema.json'}},
  'createdAt': datetime.datetime(2024, 7, 12, 22, 57, 32, 764243, tzinfo=tzlocal()),
  'description': 'APIs t

### Allowing agent to invoke action group Lambda
Before using our action group, we need to allow our agent to invoke the Lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the Lambda function created:

In [15]:
# Create allow invoke permission on Lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Associating the agent to knowledge base 1


In [16]:
agent_kb_db_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {knowledge_base_db_id} Knowledge Base to generate a valid SQLite Query to answer the questions based on the Northwind database',
    knowledgeBaseId=knowledge_base_db_id 
)

In [17]:
agent_kb_db_description

{'ResponseMetadata': {'RequestId': '7d710cf0-04fa-49e6-939b-d720014838ed',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 22:57:33 GMT',
   'content-type': 'application/json',
   'content-length': '334',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7d710cf0-04fa-49e6-939b-d720014838ed',
   'x-amz-apigw-id': 'a0kykGnhoAMEetQ=',
   'x-amzn-trace-id': 'Root=1-6691b4dc-29db7f124f2b18f7576cb523'},
  'RetryAttempts': 0},
 'agentKnowledgeBase': {'createdAt': datetime.datetime(2024, 7, 12, 22, 57, 33, 104310, tzinfo=tzlocal()),
  'description': 'Use the information in the BEQZSQZPG0 Knowledge Base to generate a valid SQLite Query to answer the questions based on the Northwind database',
  'knowledgeBaseId': 'BEQZSQZPG0',
  'knowledgeBaseState': 'ENABLED',
  'updatedAt': datetime.datetime(2024, 7, 12, 22, 57, 33, 104310, tzinfo=tzlocal())}}

### Associating the agent to knowledge base 2

In [18]:
agent_kb_aws_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description=f'Use the information in the {knowledge_base_aws_id} Knowledge Base to answer questions based on the AWS design best practices for enterprise software and AWS well architected framework',
    knowledgeBaseId=knowledge_base_aws_id 
)

In [19]:
agent_kb_aws_description

{'ResponseMetadata': {'RequestId': '874dc2bc-cc5f-4f18-b393-f2fb4f619b7b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 22:57:33 GMT',
   'content-type': 'application/json',
   'content-length': '363',
   'connection': 'keep-alive',
   'x-amzn-requestid': '874dc2bc-cc5f-4f18-b393-f2fb4f619b7b',
   'x-amz-apigw-id': 'a0kymF_7IAMEmBA=',
   'x-amzn-trace-id': 'Root=1-6691b4dd-0acd45024d69d0cd712cc91a'},
  'RetryAttempts': 0},
 'agentKnowledgeBase': {'createdAt': datetime.datetime(2024, 7, 12, 22, 57, 33, 299928, tzinfo=tzlocal()),
  'description': 'Use the information in the M9G3JNUERS Knowledge Base to answer questions based on the AWS design best practices for enterprise software and AWS well architected framework',
  'knowledgeBaseId': 'M9G3JNUERS',
  'knowledgeBaseState': 'ENABLED',
  'updatedAt': datetime.datetime(2024, 7, 12, 22, 57, 33, 299928, tzinfo=tzlocal())}}

### Preparing the agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [20]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '6f37ffbd-c008-44af-9736-8e7fb2bdb8ad',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 22:57:33 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6f37ffbd-c008-44af-9736-8e7fb2bdb8ad',
   'x-amz-apigw-id': 'a0kyoECwoAMEo2A=',
   'x-amzn-trace-id': 'Root=1-6691b4dd-287b8cba3050f1560301c5f7'},
  'RetryAttempts': 0},
 'agentId': 'WONPD0DVUU',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 7, 12, 22, 57, 33, 449601, tzinfo=tzlocal())}

### <a name="8">Create agent alias to deploy agent</a>
(<a href="#0">Go to top</a>)

We will now create an alias of the agent that can be used to deploy the agent.

In [21]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
# Pause to make sure agent alias is ready
time.sleep(30)

In [22]:
agent_alias

{'ResponseMetadata': {'RequestId': 'ab7feff7-b802-49d8-869a-ab5584b68a5e',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 12 Jul 2024 22:58:03 GMT',
   'content-type': 'application/json',
   'content-length': '349',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ab7feff7-b802-49d8-869a-ab5584b68a5e',
   'x-amz-apigw-id': 'a0k3WFlFoAMEN3g=',
   'x-amzn-trace-id': 'Root=1-6691b4fb-00a18aa4555d0076537c10ed'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:339712993987:agent-alias/WONPD0DVUU/NMFTQBL7SB',
  'agentAliasId': 'NMFTQBL7SB',
  'agentAliasName': 'l2ecb7c0-workshop-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'WONPD0DVUU',
  'createdAt': datetime.datetime(2024, 7, 12, 22, 58, 3, 632455, tzinfo=tzlocal()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 7, 12, 22, 58, 3, 632455, tzinfo=tzlocal())}}

### <a name="9">Invoke agent</a>
(<a href="#0">Go to top</a>)

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [23]:
USER_PROMPT_TEMPLATE = """Question: {question}

Given an input question, you will use the existing Knowledge Bases on AWS Well-Architected Framework and Northwind DB Knowledge Base. 

- For building and designing software applications, you will use the existing Knowledge Base on AWS well-architected framework to generate a response of the most relevant design principles and links to any documents. This Knowledge Base response can then be passed to the functions available to answer the user question. The final response to the direct answer to the user question. It has to be in markdown format highlighting any text of interest. Remove any backticks in the final response.

- To generate code for a given user question,  you can use the default Large Language model to come up with the response. This response can be in code markdown format. You can optionally provide an explanation for the code.

- To explain code for a given user question, you can use the default Large Language model to come up with the response. 

- For SQL query generation you will ONLY use the existing database schemas in the Northwind DB Knowledge Base to create a syntactically correct SQLite query and then you will EXECUTE the SQL Query using the functions and API provided to answer the question. 
Make sure to use ONLY existing columns and tables based on the Northwind DB database schema. Make sure to wrap table names with square brackets. Do not use underscore for table names unless that is part of the database schema. Make sure to add a semicolon after the end of the SQL statement generated.
Remove any backticks and any html tags like <table><th><tr> in the final response.

Here are a few examples of questions I can help answer by generating and then executing a SQLite query:

- What are the total sales amounts by year?

- What are the top 5 most expensive products? 

- What is the total revenue for each employee?



"""

### <a name="10">Code generation examples </a>
(<a href="#0">Go to top</a>)

For this demo, we query the LLM to generate code snippets for UI and back-end/middle tier. Change the question to generate the code in different programming languages like Python, JavaScript, Java and others.

<center><img src="images/agents-codecall.drawio.png" alt="architecture diagram for lab 3 to demonstrate the conditional workflow for LLMs.The same design diagram from earlier in the notebook, but we have highlighted the area that shows we will be working on code generation and explanation piece of the AI assistant." width="600" height="600" style="background-color:white; padding:1em;" /></center> <br/>



In [24]:
question = "Write a python function to validate email address syntax."

In [25]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from mlu_utils.agents_utils import *

In [26]:
%%time
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question),
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 1)

session_id :::: 43cd4dd2-40a2-11ef-9f7f-0e08ab083011


[2024-07-12 22:58:58,619] p28855 {agents_utils.py:88} INFO - Final answer ->
Here is a Python function to validate email address syntax:

```python
import re

def validate_email(email):
  """
  Validate email address syntax.

  Args:
    email (str): Email address to validate

  Returns:
    bool: True if valid email, False otherwise
  """

  email_regex = r'^w+([.-]?w+)*@w+([.-]?w+)*(.w{2,3})+$'

  if re.search(email_regex, email):
    return True
  else:
    return False
```

This uses a regular expression to check if the email matches the expected pattern for a valid email address. It returns True if the email is valid, False otherwise.


CPU times: user 31.8 ms, sys: 7.25 ms, total: 39.1 ms
Wall time: 24.8 s


In [27]:
# And here is the response if you just want to see agent's reply
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="1", gen_sql=False)

,User Question,Agent Answer
0,Write a python function to validate email address syntax.,"Here is a Python function to validate email address syntax:```pythonimport redef validate_email(email): """""" Validate email address syntax. Args: email (str): Email address to validate Returns: bool: True if valid email, False otherwise """""" email_regex = r'^w+([.-]?w+)*@w+([.-]?w+)*(.w{2,3})+\$' if re.search(email_regex, email): return True else: return False```This uses a regular expression to check if the email matches the expected pattern for a valid email address. It returns True if the email is valid, False otherwise."


In [28]:
# Deep dive into agent-workflow steps in each of the tab
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 1)
display(out_3_tabs_1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
trace_filename_prefix = lab3_agent_trace and turn_number = 1


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [29]:
# Wait for tab output async processes to complete
time.sleep(10)

In [30]:
%%time
# Using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
turn_1_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="1")
with out_3_turn_1_summary:
    out_3_turn_1_summary.clear_output()
    display(Markdown(turn_1_summary))

>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab3_agent_trace_1.log


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 117 ms, sys: 36.6 ms, total: 153 ms
Wall time: 3.81 s


In [31]:

## Display the summary of the agent workflow trace
display(out_3_turn_1_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [32]:
code_to_explain = None
with open("mlu_utils/summarize_agent_trace.py", "r") as code_fp:
    code_to_explain = code_fp.read()

with out:
    out.clear_output()
    # display first 500 characters of the py file 
    display(Markdown(code_to_explain[:450]))

display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [62]:
question = f"Explain the following code in lucid, natural language to me. \n code to explain : \n {code_to_explain}"

In [63]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']



session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question),
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 2)



session_id :::: 260606c0-40a9-11ef-9f7f-0e08ab083011


[2024-07-12 23:49:43,797] p28855 {agents_utils.py:88} INFO - Final answer ->
Here is an explanation of the provided Python code in simple terms:

The code imports several modules:

- langchain_core.output_parsers: This provides the StrOutputParser class to parse text output.

- langchain_core.prompts: This provides the ChatPromptTemplate class for generating chatbot prompts.

- langchain_community.chat_models: This provides the BedrockChat class for accessing AI chat models.

- langchain_core.messages: This provides the HumanMessage class for formatting human input messages. 

- boto3: This is the AWS SDK for Python, used to call AWS services.

It defines:

- MODEL_ID: The ID of the Claude AI model to use.

- bedrock_runtime_client: A Boto3 client for accessing the Bedrock runtime.

- model_kwargs: Parameters to configure the Claude model.

It then defines the summarize_agent_trace() function which:

- Constructs the path to an agent trace log file.

- Reads the trace file content into

In [64]:
# And here is the response if you just want to see agent's reply
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="2", gen_sql=False)

,User Question,Agent Answer
0,"Explain the following code in lucid, natural language to me using LLM. code to explain : from langchain_core.output_parsers import StrOutputParserfrom langchain_core.prompts import ChatPromptTemplatefrom langchain_community.chat_models import BedrockChatfrom langchain_core.messages import HumanMessageimport boto3MODEL_ID = ""anthropic.claude-3-haiku-20240307-v1:0""bedrock_runtime_client = boto3.client(""bedrock-runtime"")model_kwargs = { ""max_tokens"": 4000, ""temperature"": 0.0, ""top_k"": 250, ""top_p"": 1, ""stop_sequences"": [""Human""],}def summarize_agent_trace(trace_file_base_path= ""trace_files/"", lab_number=""2b"", turn_number=""1""): complete_log_path = trace_file_base_path + f""full_trace_lab{lab_number}_agent_trace_{turn_number}.log"" print(f"">>>>>>>> complete_log_path to summarize==> {complete_log_path}"") trace_content_text = None with open(complete_log_path, ""r"") as trace_fp: trace_content_text = trace_fp.read().replace("""", """") # print(f""trace_content_text[:20] == {trace_content_text[:20]}"") llm = BedrockChat(client=bedrock_runtime_client, model_id=MODEL_ID, model_kwargs=model_kwargs) messages = [HumanMessage(content=f""Summarize in natural language the agent workflow in the trace file contents that follow. Remove XML tags in the output. TRACE FILE CONTENTS: {trace_content_text[:40000]}"")] response = llm(messages) # print(f""raw response ==> {response}"") if str(type(response)) == ""<class 'langchain_core.messages.ai.AIMessage'>"": response = response.content response = response.strip() #print(f""Parsed response ==> {response}"") return response# for unit-testing# summarize_agent_trace(trace_file_base_path= ""../trace_files/"", lab_number=""2b"", turn_number=""1"")","Here is an explanation of the provided Python code in simple terms:The code imports several modules:- langchain_core.output_parsers: This provides the StrOutputParser class to parse text output.- langchain_core.prompts: This provides the ChatPromptTemplate class for generating chatbot prompts.- langchain_community.chat_models: This provides the BedrockChat class for accessing AI chat models.- langchain_core.messages: This provides the HumanMessage class for formatting human input messages. - boto3: This is the AWS SDK for Python, used to call AWS services.It defines:- MODEL_ID: The ID of the Claude AI model to use.- bedrock_runtime_client: A Boto3 client for accessing the Bedrock runtime.- model_kwargs: Parameters to configure the Claude model.It then defines the summarize_agent_trace() function which:- Constructs the path to an agent trace log file.- Reads the trace file content into a string.- Creates a BedrockChat instance using the Claude model.- Sends the trace content to Claude and asks it to summarize the agent workflow.- Parses Claude's response and returns the summary text.In summary, it provides utilities to load an agent trace file, send it to Claude, and get back a natural language summary.I apologize that I was unable to successfully call the provided functions to assist in answering this question. Please let me know if I can try again with an example function call to better learn the proper format."


In [65]:
# Deep dive into agent-workflow steps in each of the tabs
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 2)
display(out_3_tabs_2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
trace_filename_prefix = lab3_agent_trace and turn_number = 2


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [66]:
# Wait for tab output async processes to complete
time.sleep(10)

In [67]:
%%time
# Using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
turn_2_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="2")
with out_3_turn_2_summary:
    out_3_turn_2_summary.clear_output()
    display(Markdown(turn_2_summary))

>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab3_agent_trace_2.log
CPU times: user 13.9 ms, sys: 294 μs, total: 14.2 ms
Wall time: 6.29 s


In [68]:
## Display the summary of the Agent workflow trace
display(out_3_turn_2_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…


### <a name="11">System design Q&A with AWS Well-Architected Framework</a>
(<a href="#0">Go to top</a>)

For this demo, we query the LLM to recommend system design setup and best practices based on the robust AWS Well-Architected Framework. 

<center><img src="images/agents-awscall.drawio.png" alt="architecture diagram for lab 3 to demonstrate the conditional workflow for LLMs.The same design diagram from earlier in the notebook, but we have highlighted the area that shows we will be working on the AWS Well-Architected Framework for knowledge base inquiry." width="600" height="600" style="background-color:white; padding:1em;" /></center> <br/>



In [40]:
question = "How can I design secure VPCs?"

In [41]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']


session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question),
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 3)


session_id :::: 68ff619e-40a2-11ef-9f7f-0e08ab083011


[2024-07-12 23:00:12,830] p28855 {agents_utils.py:88} INFO - Final answer ->
To design secure VPCs, you should implement the following best practices:

- Plan your network topology to accommodate growth, regulatory compliance, and integration with other networks. Allocate IP addresses to subnets across Availability Zones to ensure availability as you scale (Result 1).

- Control traffic at all layers using security groups, network ACLs, VPC endpoints, AWS PrivateLink, and edge services like CloudFront and AWS WAF (Results 2 and 3).

- Implement authentication and authorization between services using VPC Lattice, mutual TLS, API Gateway, and other mechanisms (Results 4 and 5).


In [42]:
# And here is the response if you just want to see agent's reply
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="3", gen_sql=False)

,User Question,Agent Answer
0,How can I design secure VPCs?,"To design secure VPCs, you should implement the following best practices:- Plan your network topology to accommodate growth, regulatory compliance, and integration with other networks. Allocate IP addresses to subnets across Availability Zones to ensure availability as you scale (Result 1).- Control traffic at all layers using security groups, network ACLs, VPC endpoints, AWS PrivateLink, and edge services like CloudFront and AWS WAF (Results 2 and 3).- Implement authentication and authorization between services using VPC Lattice, mutual TLS, API Gateway, and other mechanisms (Results 4 and 5)."


In [43]:
# Deep dive into Agent-workflow steps in each of the tabs
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 3)
display(out_3_tabs_3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
trace_filename_prefix = lab3_agent_trace and turn_number = 3


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [44]:
# Wait for tab output async processes to complete
time.sleep(10)

In [45]:
%%time
# Using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
turn_3_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="3")
with out_3_turn_3_summary:
    out_3_turn_3_summary.clear_output()
    display(Markdown(turn_3_summary))

>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab3_agent_trace_3.log
CPU times: user 12.3 ms, sys: 305 μs, total: 12.6 ms
Wall time: 8.39 s


In [46]:
## Display the summary of the agent workflow trace
display(out_3_turn_3_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### <a name="12">SQL generation and execution against a database</a>
(<a href="#0">Go to top</a>)

For this demo, we query the LLM to generate SQL code based on a database schema of our choice and execute it against a database to validate the accuracy of the generated code.

<center><img src="images/agents-SQLcall.drawio.png" alt="architecture diagram for lab 3 to demonstrate the conditional workflow for LLMs.The same design diagram from earlier in the notebook, but we have highlighted the area that shows we will be working on the Text to SQL generation from natural language questions." width="600" height="600" style="background-color:white; padding:1em;" /></center> <br/>



In [47]:
question = "What are the total sales amounts by year?"

In [48]:
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']


## Create a random id for session initiator id
session_id:str = str(uuid.uuid1()) # random identifier
enable_trace:bool = True
end_session:bool = False

print(f"session_id :::: {session_id}")

final_answer = invoke_agent_generate_response(bedrock_agent_runtime_client,
                                               USER_PROMPT_TEMPLATE.format(question=question), 
                                               agent_id, 
                                               agent_alias_id, 
                                               session_id, 
                                               enable_trace,
                                               end_session,
                                               trace_filename_prefix = 'lab3_agent_trace',
                                               turn_number = 4)



session_id :::: 8a04a8cc-40a2-11ef-9f7f-0e08ab083011


[2024-07-12 23:01:28,038] p28855 {agents_utils.py:88} INFO - Final answer ->
The total sales amounts grouped by year are:

| Year | Total Sales |
|-|-|  
| 2012 | $18,823,201.72 |
| 2013 | $38,633,120.01 |   
| 2014 | $38,870,148.13 |
| 2015 | $41,423,456.72 |
| 2016 | $40,568,672.36 |
| 2017 | $40,209,904.23 |
| 2018 | $38,326,623.43 |
| 2019 | $38,516,963.86 |
| 2020 | $38,862,436.79 |
| 2021 | $41,355,549.74 |
| 2022 | $39,742,066.18 |
| 2023 | $33,054,490.00 |


In [49]:
# And here is the response if you just want to see agent's reply
%load_ext autoreload
%autoreload 2
from mlu_utils.agents_utils import *
format_final_response(question=question, final_answer=final_answer, lab_number="3", turn_number="4", gen_sql=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,User Question,Agent Generated SQL,Agent Answer
0,What are the total sales amounts by year?,"SELECT strftime('%Y', [OrderDate]) AS [Year], SUM([Quantity] * [UnitPrice] * (1 - [Discount])) AS TotalSales FROM [Order Details] JOIN Orders ON [Order Details].[OrderID] = [Orders].[OrderID] GROUP BY [Year] ORDER BY [Year]","The total sales amounts grouped by year are:| Year | Total Sales ||-|-| | 2012 | \$18,823,201.72 || 2013 | \$38,633,120.01 | | 2014 | \$38,870,148.13 || 2015 | \$41,423,456.72 || 2016 | \$40,568,672.36 || 2017 | \$40,209,904.23 || 2018 | \$38,326,623.43 || 2019 | \$38,516,963.86 || 2020 | \$38,862,436.79 || 2021 | \$41,355,549.74 || 2022 | \$39,742,066.18 || 2023 | \$33,054,490.00 |"


In [50]:
# Deep dive into Agent-workflow steps in each of the tabs
%load_ext autoreload
%autoreload 2
from mlu_utils.show_trace_widget import *

show_tabs(trace_filename_prefix = 'lab3_agent_trace', turn_number = 4)
display(out_3_tabs_4)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
trace_filename_prefix = lab3_agent_trace and turn_number = 4


ToggleButtons(button_style='success', description='Agent Trace: (Last Logged trace only) Full trace available …

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [51]:
# Wait for tab output async processes to complete
time.sleep(10)

In [52]:
%%time
# Using Claude-v3 Haiku to generate a summary of the agent's workflow for this conversation turn
turn_4_summary = summarize_agent_trace(trace_file_base_path= "trace_files/", lab_number="3", turn_number="4")
with out_3_turn_4_summary:
    out_3_turn_4_summary.clear_output()
    display(Markdown(turn_4_summary))

>>>>>>>> complete_log_path to summarize==> trace_files/full_trace_lab3_agent_trace_4.log
CPU times: user 13.3 ms, sys: 251 μs, total: 13.5 ms
Wall time: 5.64 s


In [53]:
## Display the summary of the agent workflow trace
display(out_3_turn_4_summary)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

### <a name="9">[Be Frugal] Clean up resources </a>
(<a href="#0">Go to top</a>)


##### In the following cell, we offer an option raise an exception to avoid auto-executing the next block of lines and optionally clean up all resources. This is useful when the `Kernel > run all` option is used.

`Please be frugal if you choose to enable this exception in the code cell below. By default it is disabled and all resources will be cleaned up immediately to avoid additional costs.`

##### Within the same kernel session, this will allow experimentation with different prompts without having to recreate agent resources (takes ~5 minutes)

In [54]:
# This avoids auto-cleanup
Raise Exception('Avoiding Auto-Cleanup of Bedrock Agent Resources')

SyntaxError: invalid syntax (2290889608.py, line 2)

In [ ]:
cleanup_infrastructure(agent_action_group_response, lambda_name, lambda_function, lambda_role_name, agent_id, agent_alias_id, agent_role_name, bucket_name, schema_key, agent_bedrock_policy, agent_s3_schema_policy, agent_kb_schema_policy, kb_db_bedrock_policy, kb_db_aoss_policy, kb_db_s3_policy, kb_db_role_name, kb_db_collection_name, kb_db_opensearch_collection_response, knowledge_base_db_id, kb_aws_bedrock_policy, kb_aws_aoss_policy, kb_aws_s3_policy, kb_aws_role_name, kb_aws_collection_name, kb_aws_opensearch_collection_response, knowledge_base_aws_id)

In [ ]:
## Cleanup S3 separately - it has 2 KBs with GBs of data
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.all().delete()

s3_client.delete_bucket(Bucket=bucket_name)
print(f"\n bucket DELETED :: {bucket_name}")


## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You